### First order differencing

In [ ]:
import pandas as pd 

rms = pd.DataFrame(rms_plot)
rms_stationary = rms.diff().values
vmin_rms = np.min(rms_stationary)
vmax_rms = np.max(rms_stationary)
print(vmin_rms)
#plotting
figure=plt.figure(figsize=(8,5), dpi=150)
mesh=plt.pcolormesh(channels_rel, times, rms_stationary, cmap='viridis',vmin=vmin_rms, vmax=vmax_rms)
cbar=plt.colorbar(mappable=mesh, label=label_cbar)
#plt.gca().invert_yaxis()
plt.xlabel("channel_rel"); plt.ylabel("Time [s]")
plt.title(f"{time_utc} UTC")
plt.show()

### Autocorrelation at lag 1

In [ ]:

import numpy as np

# Parameters
data_points_per_set = 10  # Number of data points in each time set

# Assuming your data is stored in a DataFrame named 'rms_stationary'
# 'rms_stationary' should have shape (2800, 301)

# Convert the DataFrame to a NumPy array for reshaping
rms_stationary_array = rms_stationary

# Reshape the data into time sets
num_sets = len(rms_stationary_array) // data_points_per_set
reshaped_data = rms_stationary_array[:num_sets * data_points_per_set].reshape(num_sets, data_points_per_set, -1)

# Verify the shape of the reshaped data
print("Shape of reshaped data:", reshaped_data.shape)  # This should be (number of sets, data points per set, spatial locations)




In [ ]:
import pandas as pd

# Initialize a DataFrame to store ACF at lag 1 for each time set and spatial location
acf_df = pd.DataFrame(index=range(num_sets), columns=range(reshaped_data.shape[2]))

# Calculate ACF at lag 1 for each time set and spatial location
for i in range(num_sets):
    for j in range(reshaped_data.shape[2]):
        # Calculate ACF at lag 1 for the current spatial location
        acf_df.at[i, j] = pd.Series(reshaped_data[i, :, j]).autocorr(lag=1)




import seaborn as sns
acf_df = acf_df.apply(pd.to_numeric)
acf_array = acf_df.values
acf_array = acf_array
vmin_rms = np.min(acf_array)
vmax_rms = np.max(acf_array) 
# Create a meshgrid for plotting
x, y = np.meshgrid(range(acf_array.shape[1]), range(acf_array.shape[0]))

# Plot the autocorrelation heatmap using pcolormesh
figure=plt.figure(figsize=(8,5), dpi=150)
mesh=plt.pcolormesh(channels_rel, times, acf_array, cmap='viridis',vmin=vmin_rms, vmax=vmax_rms)
cbar=plt.colorbar(mappable=mesh, label=label_cbar)
#plt.gca().invert_yaxis()
plt.xlabel("channel_rel"); plt.ylabel("Time [s]")
plt.title(f"{time_utc} UTC")
plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go


# List to store scatter plots for each configuration
picks_list = []
picks_ch1000_list = []
heatmaps_list = []

# Double loop to process each rms_plot for every parameter configuration
for i in range(len(window_sizes)):
    window_size = window_sizes[i]
    min_amplitude = min_amplitudes[i]
    eps_value = eps[i]

    # List to store processed data for each rms_plot
    picks = []
    picks_ch1000 = []
    heatmaps = []
    times_list = []
    # Process each rms_plot for the current parameter configuration
    for j in range(len(rms_plot_list)):
        rms_plot = rms_plot_list[j]
        
        rms_processed = process_strain_data(
            rms_plot,
            window_size=window_size,
            min_amplitude=min_amplitude,
            min_distance=min_distance,
            eps=eps_value,
            min_samples=min_samples
        )
        # Get the indices of non-zero values
        non_zero_indices = np.nonzero(rms_processed)
        locations = non_zero_indices[1]  # Spatial locations
        timestamps_indices = non_zero_indices[0]
        timestamps_indices_ch1000 = np.nonzero(rms_processed[:,int(map_point_between_ranges(1000, reverse = True))])
    
        marker_sizes = rms_processed[timestamps_indices, locations]

        min_size = 5
        max_size = 15
        normalized_sizes = np.interp(marker_sizes, (marker_sizes.min(), marker_sizes.max()), (min_size, max_size))
        
        start_time_utc = str(time_utc_list[j])
        step_size_seconds = dt_rms_list[j]
        num_timestamps = rms_list[j].shape[0]
        times = generate_timestamps(start_time_utc, step_size_seconds, num_timestamps)
        
        picks_ch1000.append([times[k].timestamp() for k in timestamps_indices_ch1000[0]])


        log_points = go.Scatter(
            x=map_point_between_ranges(locations),
            y=[times[k] for k in timestamps_indices],
            mode='markers',
            marker=dict(color='black',
                        size=normalized_sizes),
            name=f'Strain value of pick',
            showlegend = True if j == 0 else False
        )
        picks.append(log_points)

        heatmap = go.Heatmap(
        z=rms_plot_list[j],
        x=channels_rel,
        y=times,
        colorscale='viridis',
        colorbar=dict(x=1.0, y=0.7, len=0.4)
        )

        heatmaps.append(heatmap)

    heatmaps_list.append(heatmaps)
    picks_list.append(picks)
    picks_ch1000_list.append(picks_ch1000)